In [1]:
%run -i ../python/common.py
publish=False

if not publish:
    # cleanup any old state
    bashCmds('''[[ -d mydir ]] && rm -rf mydir
    [[ -a myfile ]] && rm myfile
    [[ -a errors ]] && rm errors 
    [[ -a mydate ]] && rm mydate
    [[ -a mynewdir ]] && rm -rf mynewdir
    [[ -a anotherfile ]] && rm anotherfile
    [[ -a mybin ]] && rm -rf  mybin
    [[ -a myinfo ]] && rm myinfo''')
else:
    bashCmds('''rm -rf ~/*''')
    
closeAllOpenTtySessions()
bash = BashSession()

generated="~/myfile ~/errors ~/mydate ~/mydir ~/mynewdir ~/out"


```{warning}
This portion of the book is under construction, not ready to be read
```

(cont:gs:abstractions)=
# Operating System Abstractions

UNIX has instilled in generations of computer scientists a basic aesthetic for how to design and structure complicated collections of software.  In particular one learns that the designers of UNIX tried to structure the system around a small core set of ideas, "abstractions", that once understood allows a programmer to understand the rest of the system and how to get things done.   

We first describe the fundamental abstraction of [files](cont:gs:abstractions:file) that is core to the power and understand of Unix's ability to enable programs, to be composed together, and how this enables powerful functionality to be implemented in a shell.  We then, using a shell as a example, briefly discuss key abstractions of [processes](cont:gs:abstractions:process), [file systems](cont:gs:abstractions:fs), [pipes](cont:gs:abstractions:pipes) and [signals](cont:gs:abstractions:signals). 


(cont:gs:abstractions:file)=
## Everything is a file

A core idea of Unix is that everything is a file, where a file is a stream of bytes.  As shown {numref}`file-desc` the kernel maintains a table of open files, and a file descriptor is an index into that table that can be used to read or write to a particular file.  The system calls that work on all files are:

- `n = write(desc, buffer, len)`: Write `len` bytes from `buffer` into a stream identified by `desc`.

- `n = read(desc, buffer, max)`: Read `max` bytes (or fewer if no data is available) from stream identified by `desc` into `buffer`and returning the actual number of bytes `n` read.  

To understand how to use these operations, you really need to read the manual.  In Linux you can find out about everything using the **man** program.  For example, ```man 2 write``` tells you everything about the ```write``` system call:
  

In [2]:
bash.run("man 2 write", height='1in')

Output(layout=Layout(border='1px solid black', height='1in', overflow_y='scroll'))

```{note}
:class: margin
In this case, the *2* refers to the section of the manual for system calls.  To find out about the different sections, you, of course, read the manual about the man command.
```



To make files in the file system look like a stream, on each read or write operation, the kernel increases a per `desc` offset by the amount of data read or written.  Turns out that this naturally matches many applications that read or write files in their entirety, and is key to enabling a common abstraction across all I/O. 

This idea, that you can read or write to any kind of I/O object, enabled programs to write a single application, and, depending on how it is launched by the shell, it could work on durable state on a disk, work interactively against input typed in by a programmer, or even work by communicating with other processes, potentially over a network.   


Entries are added to the table by operations that open or create a file or create a special file like a network connection, or a tty.  There are three special file descriptors shown in {numref}`fd-table`, that programs (and libraries) should use for input, output and errors. 

```{figure} ../images/intro/osstructure-fd.drawio.png
---
width: 50%
name: file-desc
---
The kernel maintains for each process an array of file descriptors, where a process can read or write to any kind of I/O object that are open in its table. 
```

```{list-table} Standard well known file descriptors. 
:header-rows: 1
:name: fd-table
:widths: 3 5 20
:width: 4in

* - Value
  - Name
  - Purpose
* - 0
  - stdin
  - standard input; process should read data from here
* - 1
  - stdout
  - standard output; process will write its output here
* - 2
  - stderr
  - standard error; process should write out errors here
```

So, for example, the following program echos a string to the terminal:

In [3]:
bash.run("echo \"Hello class\"", height='1in')

Output(layout=Layout(border='1px solid black', height='1in', overflow_y='scroll'))

In [4]:
bash.run("echo \"Hello class\" > /tmp/reshello")

Output(layout=Layout(border='1px solid black', height='100%', overflow_y='scroll'))

While the same program, can have its output redirected to a file, as we can see from ```cat ``` which then prints the contents of that file to the terminal:

In [5]:
bash.run('''cat /tmp/reshello''')

Output(layout=Layout(border='1px solid black', height='100%', overflow_y='scroll'))

This probably seems obvious to a modern reader, i.e., an object oriented design, where you can do the same operations on any object. However, at the time, it was a radical idea, operating systems had specialized interfaces for files with records, terminals, etc…  Moreover, by introducing the idea of a special object/file, a *pipe*, you could allow programs to be combined together to do much more powerful tasks.   The **|** symbol tells the shell to create a pipe that connects the output of one program to the input of the next program.  So, lets say we are trying to find all the programs on our computer that have anything to do with perl, the following command will list the contents of the /usr/bin/ directory send the output of that listing to a grep program that searches for the word perl, and send the output of that to a program that counts the number of lines of input it had

In [6]:
bash.run("ls /usr/bin/ | grep perl | wc -l", height='1in')

Output(layout=Layout(border='1px solid black', height='1in', overflow_y='scroll'))

Today, this has gone much further.  Linux now exposes all kinds of information through synthetic file systems, giving users and administrators massive ability to automate.  For example, in ```bash``` the shell we are using ```$$``` lets us know the ``id`` of the process we are in.  So, stealing a nice example from [jonathan](https://jappavoo.github.io/UndertheCovers/textbook/unix/shellintro.html#standard-output-and-redirection), the following command shows the 

In [7]:
bash.run('''ls -l /proc/$$/fd/{0..2}''')

Output(layout=Layout(border='1px solid black', height='100%', overflow_y='scroll'))

In [8]:
bash_pid=bash.getPid()
bash_stdout=os.path.realpath("/proc/" + bash_pid.__str__() + "/fd/1")
bash.run("file " + bash_stdout)

Output(layout=Layout(border='1px solid black', height='100%', overflow_y='scroll'))

And we can see that our stdin, stdout, and stderr all point to a character special file is in Unix is used to represent a terminal, and we can write to that same special device and it will appear in our terminal. 

In [9]:
bash.run("echo \"hello class\" > " + bash_stdout)

Output(layout=Layout(border='1px solid black', height='100%', overflow_y='scroll'))

I would strongly encourage reading the shell and unix sections of [Under the Covers: The Secret Life of Software](https://jappavoo.github.io/UndertheCovers/textbook/intro_tb.html#under-the-covers-the-secret-life-of-software)  for a much more detailed coverage of this material.  However, hopefully this has given you enough information to understand the power Unix introduced by introducing polymorphism in the operating system, and creating a shell that enables you to combine all kinds of programs together in complicated ways.  

The remainder of this chapter introduces the core abstractions of Unix, and the system calls you use on those examples, all with examples from a shell. 

(cont:gs:abstractions:process)=
## Process management

We have already introduced the abstraction of a process, it is a virtual computer.

- new architecture picture with a process descriptor, pointing to file table, and mm data structures, P1, then P2 created by fork
- show fork and exec system calls; and say how they are so stupid

- show a simple shell, reads a program name from stdin, forks, execs, cool.  

### Changing stdin/stdout 

Introduce dup2, show how that works

### File system system calls

So, we know how to read and write from stdin, but how do we direct into a file.  

Read and write work for all files, there are also a number of system calls specific to file systems. While we discuss this in more detail [later](cont:fs:interface), we briefly introduce the key information you need to know.  First, it is important to realize that all Unix file systems organize information in a hierarchy as shown in numref}`fs:tree-logical-abs`.  

```{figure} ../images/pb-figures/fs/filesys-tree.png
---
width: 45%
name: fs:tree-logical-abs
---
Logical view: hierarchical file system name space
```

Some of the system calls specific to file systems are:

- `int desc = open(name, O_READ)`: Verify that file `name` exists and may
be read, and then return a *descriptor* which may be used to refer to
that file when reading it.

- `int desc = open(name, O_WRITE | flags, mode)`: Verify permissions and
open `name` for writing, creating it (or erasing existing contents) if
necessary as specified in `flags`. Returns a descriptor which may be
used for writing to that file.

- `close(desc)`: stop using this descriptor, and free any resources
allocated for it.

- `lseek(desc, offset, flag)`: Set an open file's current position to that
specified by `offset` and `flag`, which specifies whether `offset` is
relative to the beginning, end, or current position in the file.


Show new picture of shell code, with redirection

(cont:gs:abstractions:pipes)=
## Pipes

Okay, but remember that idea of pipe we talked about, well, .. 


(cont:gs:abstractions:signals)=
## Signals 

What if you don't want to wait for the program to finish.  Turns out... introduce background.

talk about zombies

Well remember, we have the abstraction of a virtual computer.  For the physical compter, we have the idea of interrupts.  For virtual comptuer, signals do the same thing. 

## Conclusion

You might think you know it all, but, we have only talked about 

